<a href="https://colab.research.google.com/github/sk-panigrahi-559/DataScienceNotes/blob/main/DataScienceLessons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Notes from - Data Science from Scratch</h1>
<h2>Introduction - Data Scientist at "Data Sciencester"</h2>
DataSciencester is a social network for data scientists. And the author wants us to pretend that we are hired by the company.

the following is a list of users with their user ID and names. And the next one is a list of friends connections.

In [ ]:
users = [
{ "id": 0, "name": "Hero" },
{ "id": 1, "name": "Dunn" },
{ "id": 2, "name": "Sue" },
{ "id": 3, "name": "Chi" },
{ "id": 4, "name": "Thor" },
{ "id": 5, "name": "Clive" },
{ "id": 6, "name": "Hicks" },
{ "id": 7, "name": "Devin" },
{ "id": 8, "name": "Kate" },
{ "id": 9, "name": "Klein" }
]

In [ ]:
friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
(4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

This is not a very easy to handle the connections this way. So we decided to prepare a list.

*This is the point in the book I realised I have to get used to the one liners...

In [ ]:
# Initializing the dictionary
friendships = {user["id"]:[] for user in users}
# In the book the above one liner is used one can also do:
# friendships ={}
# for user in users:
#   friendships[user["id"]] = []

for i, j in friendship_pairs:
  friendships[i].append(j)
  friendships[j].append(i)

# Now we have all the friendship connection in a dictionary


Now since we have this dictionary we can do a lot of things with it; Ex:

In [ ]:
# No. of friends a user has
def number_of_friends(user):
  user_id = user["id"]
  friend_ids = friendships[user_id]
  return len(friend_ids)

# Total no. of connections
total_connections = sum(number_of_friends(user) for user in users)

# Average number of connections
avg_connections = total_connections / len(users)

# Also most connected people - largest number of friends

Now Just like facebook, we want the users to know the friends of their friends; so we write the followig code to know that.

In [ ]:
def foaf_ids(user):
  return [foaf_id for friend_id in friendships[user["id"]] for foaf_id in friendships[friend_id]]

but if we try to get it for the users[0] what do we see?

In [ ]:
foaf_ids(users[0])

[0, 2, 3, 0, 1, 3]

Redundant entries and also it includes 1 and 2 which are already friends of 0. which doesn't look so usefull of a data.

Abetter code will be:

In [ ]:
# so first we want to create a code for mutual friends
from collections import Counter

def friends_of_friends(user):
  user_id = user["id"]
  return Counter(
      foaf_id
      for friend_id in friendships[user_id]
      for foaf_id in friendships[friend_id]
      if foaf_id != user_id and foaf_id not in friendships[user_id]
  )

print(friends_of_friends(users[3]))

Counter({0: 2, 5: 1})


This clearly says that 3 has two mutual friends with 0 and one mututal friend with 5.

Now we are interested to know interests of the users so that we can suggest connection between people with similar interests. Let's say following is a list of interests of the users.

In [ ]:
interests = [
(0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
(0, "Spark"), (0, "Storm"), (0, "Cassandra"),
(1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
(1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
(2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
(3, "statistics"), (3, "regression"), (3, "probability"),
(4, "machine learning"), (4, "regression"), (4, "decision trees"),
(4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
(5, "Haskell"), (5, "programming languages"), (6, "statistics"),
(6, "probability"), (6, "mathematics"), (6, "theory"),
(7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
(7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
(8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
(9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

now we create a function to find out people with same interests

In [ ]:
def data_scientists_who_like(target_interest):
  """Find the ids of all users who like the target interest."""
  return [user_id
    for user_id, user_interest in interests
    if user_interest == target_interest]

This works, but it has to examine the whole list of interests for every
search. If we have a lot of users and interests (or if we just want to do a lot
of searches), we’re probably better off building an index from interests to
users:

In [ ]:
from collections import defaultdict

# Keys are interests, values are lists of user_ids with that interest
user_ids_by_interest = defaultdict(list)
for user_id, interest in interests:
  user_ids_by_interest[interest].append(user_id)

# Keys are user_ids, values are lists of interests for that user_id.
interests_by_user_id = defaultdict(list)
for user_id, interest in interests:
  interests_by_user_id[user_id].append(interest)

# and hence find the people with highest common interests
def most_common_interests_with(user):
  return Counter(
    interested_user_id
    for interest in interests_by_user_id[user["id"]]
    for interested_user_id in user_ids_by_interest[interest]
    if interested_user_id != user["id"]
  )

---
<h2>* Some Python things till now that I didn't know</h2>



<h3>1. defaultdict</h3>
Imagine that you’re trying to count the words in a document. An obvious
approach is to create a dictionary in which the keys are words and the
values are counts. As you check each word, you can increment its count if
it’s already in the dictionary and add it to the dictionary if it’s not:

In [ ]:
document = "This is some document opened in the reading mode and .read() function was used on that"
word_counts = {}
for word in document:
  if word in word_counts:
    word_counts[word] += 1
  else:
    word_counts[word] = 1

another approach is

In [ ]:
word_counts = {}
for word in document:
  previous_count = word_counts.get(word, 0)
  word_counts[word] = previous_count + 1

Every one of these is slightly unwieldy, which is why defaultdict is
useful. A defaultdict is like a regular dictionary, except that when you try
to look up a key it doesn’t contain, it first adds a value for it using a zero-argument function you provided when you created it. In order to use
defaultdict s, you have to import them from collections:

In [ ]:
from collections import defaultdict
word_counts = defaultdict(int)
 # int() produces 0
for word in document:
  word_counts[word] += 1

They can also be useful with list or dict, or even your own functions:

In [ ]:
dd_list = defaultdict(list)               # list() produces an empty list
dd_list[2].append(1)                      # now dd_list contains {2: [1]}
dd_dict = defaultdict(dict)               # dict() produces an empty dict
dd_dict["Joel"]["City"] = "Seattle"       # {"Joel" : {"City": Seattle"}}
dd_pair = defaultdict(lambda: [0, 0])
dd_pair[2][1] = 1                         # now dd_pair contains {2: [0, 1]}

<h3>2. Counters</h3>
A Counter turns a sequence of values into a defaultdict(int)-like object
mapping keys to counts:

In [ ]:
from collections import Counter
c = Counter([0, 1, 2, 0])         # c is (basically) {0: 2, 1: 1, 2: 1}

This gives a very simple way to solve the word count problem

In [ ]:
# recall, document is a list of words
word_counts = Counter(document)

A Counter instance has a most_common method that is frequently useful:

In [ ]:
# print the 10 most common words and their counts
for word, count in word_counts.most_common(10):
  print(word, count)

<h3>3. Sets</h3>
Another useful data structure is set, which represents a collection of distinct
elements. You can define a set by listing its elements between curly braces:

In [ ]:
primes_below_10 = {2, 3, 5, 7}

However, that doesn’t work for empty sets, as {} already means “empty
dict.” In that case you’ll need to use set() itself:

In [ ]:
s = set()
s.add(1)
s.add(2)
s.add(2)
x = len(s)
y = 2 in s
z = 3 in s

We’ll use sets for two main reasons. The first is that in is a very fast
operation on sets. If we have a large collection of items that we want to use
for a membership test, a set is more appropriate than a list:

In [ ]:
hundreds_of_other_words = ["This will have a list of all stopwords"]
stopwords_list = ["a", "an", "at"] + hundreds_of_other_words + ["yet", "you"]
"zip" in stopwords_list
 # False, but have to check every element
stopwords_set = set(stopwords_list)
"zip" in stopwords_set
 # very fast to check

The second reason is to find the distinct items in a collection:

In [ ]:
item_list = [1, 2, 3, 1, 2, 3]
num_items = len(item_list)
item_set = set(item_list)
num_distinct_items = len(item_set)
distinct_item_list = list(item_set)

print("number of items in the list:", num_items)
print("number of distint items in the list", num_distinct_items)